In [12]:
# Import Libraries

import pandas as pd
import numpy as np
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import os
import nltk
# import pycountry
import re
import string

# from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
twitter_tweets = pd.read_csv("./twitter_tweets_final_1.csv")
twitter_tweets.head()

,UserName,Handle,Timestamp,Text,Emojis,Comments,Likes,Retweets
0,Hosky Token ($HOSKY),@hoskytoken,2021-11-18T01:26:13.000Z,I present you the King of all Idiots! \r\n \r\...,NaN,30,34,212
1,Hosky Token ($HOSKY),@hoskytoken,2021-11-18T01:14:44.000Z,"$HOSKY Token, forcing others to innovate as we...",NaN,18,17,203
2,CONR₳D,@conraddit,2021-11-18T00:52:55.000Z,We are the first generation to vote on a block...,NaN,14,26,294
3,Hosky Token ($HOSKY),@hoskytoken,2021-11-17T23:37:38.000Z,That’s a lot of soiled wallets. \r\n \r\n #car...,NaN,21,27,183
4,Healthy Pockets,@healthy_pockets,2021-11-17T23:19:16.000Z,ÚLTIMO\r\n \r\n #Cardano llega a los cajeros e...,NaN,33,144,743


In [37]:
# parameters
positive = 0
negative = 0
neutral = 0
polarity = 0

neutral_list = []
negative_list = []
positive_list = []

# tweet list from the twitter data
tweets_text = twitter_tweets["Text"]

for tweet in tweets_text:
    analysis = TextBlob(tweet)
    polarity += analysis.sentiment.polarity

    score = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']

    if neg > pos:
        negative_list.append(tweet)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet)
        positive += 1
    elif pos == neg:
        neutral_list.append(tweet)
        neutral += 1

positive_percent = 100 * float(positive) / float(len(tweets_text))
negative_percent = 100 * float(negative) / float(len(tweets_text))
neutral_percent = 100 * float(neutral) / float(len(tweets_text))
polarity_percent = 100 * float(polarity) / float(len(tweets_text))

positive_percent = format(positive_percent, '.1f')
negative_percent = format(negative_percent, '.1f')
neutral_percent = format(neutral_percent, '.1f')


In [36]:
print(positive + " (" + positive + "%) of tweets include positive sentiment")
print(negative + " (" + negative + "%) of tweets include negative sentiment")
print(neutral + " (" + neutral + "%) of tweets include neutral sentiment")

61.9 (61.9%) of tweets include positive sentiment
11.3 (11.3%) of tweets include negative sentiment
26.8 (26.8%) of tweets include neutral sentiment
